# Experiment 2-6
- `DAYS_EMPLOYED` ==  365243 -> 0
  - `0.7600575656819766` -> `0.7773064081582921`
> 성능이 떨어졌다.. 왜...?

In [1]:
from urllib.request import urlretrieve

urlretrieve('https://drive.google.com/uc?export=download&id=1XLVFI_sK0smRVVuT8XU2s-M3lJT-68sN', './open.zip')

('./open.zip', <http.client.HTTPMessage at 0x7fac82b6b850>)

In [2]:
!unzip ./open.zip

Archive:  ./open.zip
   creating: open/
  inflating: open/train.csv          
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA

In [16]:
train = pd.read_csv('./open/train.csv')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

## Data Preprocessing

In [17]:
train.fillna('NAN', inplace=True)

In [18]:
train.drop('index', axis=1, inplace=True)

In [19]:
train.drop('FLAG_MOBIL', axis=1, inplace=True)

In [20]:
binary_col = ['gender',
              'car',
              'reality']

binary_encoder = OrdinalEncoder(categories=[['F', 'M'],
                                            ['N', 'Y'],
                                            ['N', 'Y']],
                                dtype=np.int8)\
                              .fit(train.loc[:,binary_col])

train.loc[:,binary_col] = binary_encoder.transform(train.loc[:,binary_col])

In [21]:
onehot_col = ['income_type',
              'edu_type',
              'family_type',
              'house_type',
              'occyp_type']


onehot_encoder = OneHotEncoder()
onehot_encoder.fit(train.loc[:,onehot_col])


train_onehot_df = pd.DataFrame(onehot_encoder.transform(train.loc[:,onehot_col]).toarray(), 
             columns=onehot_encoder.get_feature_names(onehot_col))
train.drop(onehot_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [22]:
train['parent'] = train['family_size'] - train['child_num']

In [23]:
family_size_cut = 6
child_num_cut = 4

train.loc[train['family_size'] > family_size_cut, 'family_size'] = family_size_cut
train.loc[train['child_num'] > child_num_cut, 'child_num'] = child_num_cut

In [24]:
pca_col = ['family_size', 'child_num', 'parent']
pca = PCA(n_components=1).fit(train.loc[:, pca_col])

train['fcp-pca'] = pca.transform(train.loc[:, pca_col])
train.drop(pca_col, axis=1, inplace=True)

In [25]:
train['income_total'] = np.log1p(train['income_total'])

In [26]:
X_train = train.drop('credit', axis=1)
y_train = train['credit']

In [27]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 51 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   gender                                  26457 non-null  int64  
 1   car                                     26457 non-null  int64  
 2   reality                                 26457 non-null  int64  
 3   income_total                            26457 non-null  float64
 4   DAYS_BIRTH                              26457 non-null  int64  
 5   DAYS_EMPLOYED                           26457 non-null  int64  
 6   work_phone                              26457 non-null  int64  
 7   phone                                   26457 non-null  int64  
 8   email                                   26457 non-null  int64  
 9   begin_month                             26457 non-null  float64
 10  income_type_Commercial associate        26457 non-null  fl

## 6 `DAYS_EMPLOYED`
- `DAYS_EMPLOYED` ==  365243 -> 0
  - `0.7600575656819766` -> `0.7773064081582921`
- 0>=, 0<

In [46]:
X_train_exp = X_train.copy()

In [47]:
X_train_exp.loc[X_train_exp['DAYS_EMPLOYED']>0, :] = 0

In [48]:
X_train_exp['DAYS_EMPLOYED'].value_counts()

 0        4438
-401        57
-1539       47
-200        45
-2087       44
          ... 
-2132        1
-1452        1
-4305        1
-4401        1
-12278       1
Name: DAYS_EMPLOYED, Length: 3470, dtype: int64

In [49]:
scores = cross_validate(LGBMClassifier(), X_train_exp, y_train,
                        scoring='neg_log_loss', return_train_score = True)
scores

{'fit_time': array([0.95966744, 0.99278998, 0.9530642 , 0.94691467, 0.93616319]),
 'score_time': array([0.05926394, 0.0601449 , 0.05898118, 0.05879855, 0.06117535]),
 'test_score': array([-0.77676208, -0.78068952, -0.77925625, -0.7784826 , -0.77134158]),
 'train_score': array([-0.70046657, -0.69400548, -0.69148548, -0.69362118, -0.69318227])}

## validate

In [28]:
scores = cross_validate(LGBMClassifier(), X_train_exp, y_train,
                        scoring='neg_log_loss', return_train_score = True)
scores

{'fit_time': array([0.95952463, 0.91084599, 0.94872451, 0.92379212, 0.92964077]),
 'score_time': array([0.05964589, 0.06484652, 0.0720067 , 0.06199479, 0.06092525]),
 'test_score': array([-0.75801923, -0.76170773, -0.76034956, -0.76089536, -0.75931595]),
 'train_score': array([-0.67798479, -0.67827494, -0.67451047, -0.6794452 , -0.6779453 ])}

In [29]:
np.mean(scores['test_score'])

-0.7600575656819766